![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 2: Basic Chisel

#### Written by Stevo Bailey, Edward Wang, and Richard Lin
[stevo@berkeley.edu](mailto:stevo@berkeley.edu), [edwardw@berkeley.edu](mailto:edwardw@berkeley.edu), [richard.lin@berkeley.edu](mailto:edwardw@berkeley.edu)

## Introduction

## Table of Contents

**[Hello World: A Simple Example](#hello)**
1. [A Simple Module](#module1)
1. [A Simple Tester](#tester)
1. [Understanding Chisel vs Scala](#chiselvscala1)
1. [A Quick Debugging Primer](#debug1)

**[Basic Chisel](#chisel)**
1. [Combinational Logic](#comb)
1. [Sequential Logic](#seq)
1. [Conditional Logic and Wires](#cond)
1. [Modules](#module2)
1. [Now You Try: The Full LFSR](#lfsr)
1. [A Few More Constructs](#more)
1. [Chisel vs Scala Revisited](#chiselvscala2)
1. [Chisel Motivation as Verilog++](#motive)
1. [Debugging Tips](#debug2)

## Setup

Download and import Chisel by running this cell:

In [ ]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

// A wrapper function that provides a easy way to get the generated Verilog code for a Chisel module.
// Don't worry about the implementation of this.
def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

# Hello, World: A Simple Example<a name="hello"></a>
This section will present the simple hardware module, a simple test, and how to run it. It will contain many things that you will not understand, and that is ok. We want you to take away the broad strokes, so you can continually return to this complete working example to reinforce what you've learned.

## A Simple Module<a name="module1"></a>

Like Verilog, we can declare module definitions in Chisel. The following example is a Chisel `Module`, `Simple` that has one input, `in`, and one output, `out`, and inside it combinationally connects `in` and `out`, so `in` drives `out`.

In [ ]:
// Chisel Code: Declare a new module definition
class Simple extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

There's a lot here! The following explains how to think of each line in terms of the hardware we are describing.

```scala
class Simple extends Module {
```
We declare a new module called Simple.

```scala 
val io = IO(...)
```
We declare all our input and output ports into a special io variable.

```scala
new Bundle {
    val in = Input(...)
    val out = Output(...)
}
```
We declare a new hardware struct type that contains some named and directioned signals `in` and `out`.

```scala
UInt(4.W)
```
We declare a signal's hardware type. In this case, it is an unsigned integer of width 4.

```scala
io.out := io.in
```
We connect our input port to our output port, such that `io.in` *drives* `io.out`. Note that the `:=` operator is a ***Chisel*** operator that indicates the right-hand signal drives the left-hand signal. It is a directioned operator.

What is neat about HCL's is we can use the underlying programming language as a scripting language. For example, after declaring our Chisel module, we then use Scala to call the Chisel compiler to translate Chisel `Simple` into Verilog `Simple`. This process is called ***elaboration***.

In [ ]:
// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code; it is very complicated Scala
println(getVerilog(new Simple))

Note that the Name of our module is `cmd<#>WrapperHelperSimple`, which is an artifact of running this tutorial in Jupyter. In your normal code, it should have the name Simple. This is an important lesson though - although Chisel does its best to preserve the names of your modules and other hardware components, sometimes it fails to do so.

If we apply what we learned about Scala to this example, we can see that a Chisel module is implemented as a Scala class. Just like any other Scala class, we could make a Chisel module take some construction parameters. In this case, we make a new class `SimpleGenerator` which will accept an integer `width` that dictates the widths of its input and output ports:

In [ ]:
// Chisel Code, but pass in a parameter to set widths of ports
class SimpleGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

// Let's now generate different Simple modules with different widths
println(getVerilog(new SimpleGenerator(10)))
println(getVerilog(new SimpleGenerator(20)))

Let's dig into how this works. Because Chisel Modules are normal Scala classes, we can use the power of Scala's class constructors to parameterize the elboration of our design.

You may notice that this parameterization is enabled by *Scala*, not *Chisel*; Chisel has no extra API's for parameterization, but a design can simply leverage Scala features to parameterize their designs.

Because `SimpleGenerator` no longer describes a single Module, but instead describes a family of modules parameterized by `width`, we refer to `Simple` as a ***generator***.

## A Simple Tester <a name="tester"></a>

No hardware module or generator should be complete without a tester. The following example is a Chisel test harness that passes values to an instance of `Simple`'s input port `in`, and checks that the same value is seen on the output port `out`.

There is a ***LOT*** of crazy Scala going on here, so no need to understand anything except the `poke` and `peek` commands. You can think of the rest of the code as simply boilerplate to write these simple tests.

In [ ]:
// Scala Code: Calling Driver to instantiate Simple of width 10, SimpleTester, and execute the test
// Don't worry about understanding this code, it is very complicated Scala.
// Think of it more as boilerplate to run a Chisel tester
val testResult = Driver(() => new Simple()) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)     // Set our input to value 0
    expect(c.io.out, 0)  // Assert that the output correctly has 0
    poke(c.io.in, 1)     // Set our input to value 1
    expect(c.io.out, 1)  // Assert that the output correctly has 1
    poke(c.io.in, 2)     // Set our input to value 2
    expect(c.io.out, 2)  // Assert that the output correctly has 2
  }
}
assert(testResult)   // Scala Code: if testResult == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

What's going on? The test accepts a `Simple` module that it can assign values to its inputs, and check its outputs. To set an input, we call `poke`. To check an output, we call `expect`.

If all `expect` statements are true, then our boilerplate code will return true (see `testResult`).

<span style="color:blue">**Exercise:**</span> Write and execute two tests, one that tests SimpleGenerator for a width of 10, and a second that tests SimpleGenerator for a width of 20.

In [ ]:
val test10result = ???

val test20result = ???

assert((test10result == true) && (test20result == true))
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

### Looking at generated Verilog/FIRRTL

If you are having trouble understanding the generated hardware and are comfortable with reading structural Verilog and/or FIRRTL (Chisel's IR which is comparable to a synthesis-only subset of Verilog), then you can try looking at the generated Verilog to see the result of executing the Chisel.

Here is an example below:

In [ ]:
// A sample counter module for demonstration purposes.
class MyCounter extends Module {
  val io = IO(new Bundle {
        val in = Input(UInt(10.W))
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)
  myReg := myReg + 1.U
  io.out := myReg

  // printing results
  printf("out=%d\n", io.out)
  // The "p" below is a Scala string interpolator.
  // Chisel provides this interpolator for your convenience in printing out signals for debugging.
  printf(p"${io}\n")
}

We can view the resulting generated Verilog as shown below.

In [ ]:
// Viewing the Verilog for debugging
println(getVerilog(new MyCounter))

FIRRTL is the immediate representation (IR) of Chisel, which represents the circuit graph before it is emitted to Verilog. If you are already familiar with what FIRRTL is, then you can use it as a debugging technique as follows:

Don't worry if you don't understand this - it is not required to use Chisel.

In [ ]:
// Viewing the firrtl for debugging
println(chisel3.Driver.emit(() => new MyCounter))